In [1]:
import tensorflow as tf 
from tensorflow import keras 
import tensorflow_addons as tfa
import tensorflow_hub as hub 
from sklearn.metrics import mean_absolute_error 
import numpy as np 
import datetime

c:\Users\21265\anaconda3\envs\tfpy39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Load data

In [2]:
train_ds = tf.data.experimental.load('./data/fullscene/train_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

valid_ds = tf.data.experimental.load('./data/fullscene/valid_ds') \
    .cache().shuffle(buffer_size=1000, seed=42).prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds, valid_ds

(<PrefetchDataset shapes: ((None, 10, 224, 224, 3), (None, 5)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 10, 224, 224, 3), (None, 5)), types: (tf.float32, tf.float32)>)

### Load VGG16

In [2]:
vgg16 = keras.applications.VGG16(include_top=False, weights='imagenet', pooling='max')
vgg16.trainable = False

### Load Vit-B16

In [ ]:
vit_extractor = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-fe/versions/1/", trainable=False) # ../../vit_b16_fe/


## Build scene model

In [3]:
inputs = keras.layers.Input(shape=(10,224,224,3), name='Input')

# First model 
x      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./255.0), name='Rescaling')(inputs)
x      = keras.layers.TimeDistributed(vgg16, name='vgg16')(x)
x      = keras.layers.LSTM(units=128, return_sequences=True)(x)
x      = keras.layers.LSTM(units=64)(x)
x      = keras.layers.Dropout(0.2)(x)

x      = keras.layers.Dense(units=1024)(x)
x      = keras.layers.Dense(units=512, activation='relu')(x)


# Second model 
y      = keras.layers.TimeDistributed(keras.layers.Rescaling(scale=1./127.5, offset=-1))(inputs)

#vit_extractor = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/frameworks/TensorFlow2/variations/vit-b16-fe/versions/1/", trainable=False) # ../../vit_b16_fe/
vit_extractor.compute_output_shape = lambda x: (x[0], 768)

y      = keras.layers.TimeDistributed(vit_extractor)(y)
y      = keras.layers.LSTM(units=128, return_sequences=True)(y)
y      = keras.layers.LSTM(units=64)(y)
y      = keras.layers.Dropout(0.2)(y)
y      = keras.layers.Dense(units=1024)(y)
y      = keras.layers.Dense(units=512, activation='relu')(y)


# Averaget two models
z      = keras.layers.Average()([x,y])

z      = keras.layers.Dense(256, activation='relu')(z)
z      = keras.layers.Dropout(0.5)(z)

z      = keras.layers.Dense(5, activation='sigmoid')(z)

model  = keras.models.Model(inputs=inputs, outputs=z)
keras.utils.plot_model(model, show_shapes=True)

### Compile model

In [4]:
t = datetime.datetime.now().strftime("%m%d_%H%M%S")

optimizer      = tfa.optimizers.RectifiedAdam()

early_stopping = keras.callbacks.EarlyStopping(patience=10, verbose=0)
check_point    = keras.callbacks.ModelCheckpoint(filepath='./weights/scene/'+str(t)+'/scene.t5',
                             monitor='val_mae',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=True,
                             verbose=0)

model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

### Train

In [7]:
history = model.fit(train_ds, validation_data=valid_ds, batch_size=32, epochs=100, callbacks=[early_stopping, check_point])

Epoch 1/100
188/188 [==============================] - 353s 1s/step - loss: 0.0199 - mae: 0.1132 - val_loss: 0.0155 - val_mae: 0.0988
Epoch 2/100
188/188 [==============================] - 217s 1s/step - loss: 0.0158 - mae: 0.1006 - val_loss: 0.0158 - val_mae: 0.1012
Epoch 3/100
188/188 [==============================] - 216s 1s/step - loss: 0.0135 - mae: 0.0928 - val_loss: 0.0134 - val_mae: 0.0923
Epoch 4/100
188/188 [==============================] - 216s 1s/step - loss: 0.0117 - mae: 0.0860 - val_loss: 0.0128 - val_mae: 0.0901
Epoch 5/100
188/188 [==============================] - 215s 1s/step - loss: 0.0109 - mae: 0.0836 - val_loss: 0.0129 - val_mae: 0.0906
Epoch 6/100
188/188 [==============================] - 216s 1s/step - loss: 0.0099 - mae: 0.0794 - val_loss: 0.0132 - val_mae: 0.0913
Epoch 7/100
188/188 [==============================] - 215s 1s/step - loss: 0.0096 - mae: 0.0777 - val_loss: 0.0130 - val_mae: 0.0909
Epoch 8/100
188/188 [==============================] - 215s 1s

### Load weights

In [5]:
model.load_weights('./weights/scene/0225_123154/scene.t5')


## Evaluation

### Validation data

In [6]:
valid_ds = tf.data.experimental.load('./data/fullscene/valid_ds')
valid_ds

loss, mae = model.evaluate(valid_ds)
(1-mae)*100

63/63 [==============================] - 116s 1s/step - loss: 0.0122 - mae: 0.0878


91.21540263295174

In [7]:
y_true = np.concatenate([y for x,y in valid_ds])
y_pred = model.predict(valid_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae))*100

(array([91.1606 , 91.59788, 91.07107, 91.43924, 90.80823], dtype=float32),
 91.21540412306786)

### Test data

In [8]:
test_ds = tf.data.experimental.load('./data/fullscene/test_ds/')
loss, mae = model.evaluate(test_ds)
(1-mae)*100

63/63 [==============================] - 56s 888ms/step - loss: 0.0122 - mae: 0.0881


91.19265154004097

In [9]:
y_true = np.concatenate([y for x,y in test_ds], axis=0)
y_pred = model.predict(test_ds)

mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
(1-mae)*100, (1-np.mean(mae)) * 100

(array([91.00579, 91.68425, 91.14546, 91.14734, 90.9804 ], dtype=float32),
 91.19264855980873)